<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community
!sudo apt-get install ghostscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 277.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.5/288.5 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume BA 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/Resume baa.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume_LLM/resume 1.pdf']


## **Related Feild**

In [ ]:
resume_prompt_en = """
You are a highly experienced CEO with a reputation for demanding excellence. Your company only hires the best of the best, and you approach each resume with a critical eye, carefully weighing the value of each candidate.
As you review the resume below for a position within the Technical Labs team at Google, consider whether this candidate truly stands out in a competitive field.

**Job Description:**
- Lead key initiatives and projects within the team.
- Collaborate with cross-functional teams (product, engineering, research) to evaluate new technologies and drive business impact.
- Stay up-to-date with the latest technology trends and developments.
- Conduct research to develop innovative strategies and solutions.
- Support product development and idea validation.
- Work with experts and engineers to create proofs of concept.
- Contribute to professional communities through writing, speaking, and knowledge sharing.

**Qualifications:**
- Proven experience in strategy and execution in areas like AI, data science, business analysis, or engineering.
- Hands-on skills in relevant technologies and tools (e.g., Python, SQL, machine learning, data modeling).
- Experience applying technology to solve real-world problems.
- Strong programming or analytical skills (Python, SQL, R, etc.).
- Excellent problem-solving and communication skills.
- Ability to work in a team, prioritize tasks, and manage time effectively.
- Quick learner with a self-starter attitude.
- Knowledge in one or more industrial domains is a plus.

Resume: {resume}

**Evaluation:**

Score: [Your score here]
Strengths: [Identify the strengths of the resume]
Areas for Improvement: [Identify areas where the resume could be improved]
Overall Impression: [Summarize your overall thoughts on the resume]

**Scoring Scale:**

1-2: Very Poor – Significant gaps in skills or experience.
3-4: Poor – Some relevant skills but not meeting expectations.
5-6: Average – Meets basic requirements but lacks standout qualities.
7-8: Good – Strong qualifications but not exceptional.
9-10: Excellent – Outstanding and exceeds expectations.
"""

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient


# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        # Ensure the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Command to call Ghostscript
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",   # Output as PNG images
            f"-r{dpi}",           # Set the resolution (DPI)
            "-o", output_format,  # Output format
            pdf_file              # Input PDF file
        ]

        subprocess.run(gs_command, check=True)

        # Collect the generated image file paths
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files

    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)  # Enable GPU if available for faster processing
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])  # Sort based on y-coordinate
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to tag and clean the extracted text
def tag_and_clean_text(text):
    tagger = NamedEntityTagger()  # No device argument, as it uses GPU automatically if available
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    try:
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()
    except Exception as e:
        print(f"Error in tag_and_clean_text: {e}")
        return "", text

# Function to process a single PDF
def process_single_pdf(pdf_file):
    results = []
    try:
        # Convert PDF to images using Ghostscript
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        # Tag and clean text
        tagged_text, cleaned_text = tag_and_clean_text(raw_text)

        # Generate evaluation and score
        evaluation = generate_answer(cleaned_text)
        score = extract_score(evaluation)

        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text,
            "Evaluation": evaluation,
            "Score": score
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

def generate_answer(resume):
    formatted_prompt = resume_prompt_en.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def extract_score(text):
    match = re.search(r'Score:\s*([0-9]+(?:\.[0-9]+)?)', text)
    if match:
        return float(match.group(1))
    return text

# Batch processing with threading
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    result = generate_answer(resume)
    return result

# Define the batch processing function
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)  # Initialize a list to store results in original order
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)  # Store the index range for each future

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Gradio Interface Function
def gradio_interface(pdf_files):
    try:
        df = process_multiple_pdfs(pdf_files)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Score'] = df['Evaluation'].apply(extract_score)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Gradio UI
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Files(label="Upload PDF Files")],  # Allow multiple files
    outputs=[
        gr.Dataframe(label="Extracted and Cleaned Data", height=500, min_width=800),
        gr.File(label="Download CSV")  # Add a file download button
    ],
    title="PDF to Text and Data Cleaner"
)

# Launch the interface
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://282349faec8390ff9b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://282349faec8390ff9b.gradio.live


> 16 mins

In [ ]:
import pandas as pd
check = pd.read_csv('/content/output.csv')
check

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...,Evaluation:\n\nScore: 6\nStrengths: \n- The ca...,6.0
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,Score: 3\n\nStrengths: \nThe candidate has exp...,3.0
2,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n ประสบการณ์การทำงา...,Score: 6\n\nStrengths:\n- The candidate has a ...,6.0
3,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,**Evaluation:**\n\nScore: 5\nStrengths: \n- Th...,5.0
4,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...,11-12: Exceptional – Demonstrates exceptional ...,3.0
5,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,Assistant: **Evaluation:**\n\nScore: 5\nStreng...,5.0
6,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...,11-12: Exceptional – Rarely seen skills and ac...,8.0
7,resume 3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume\nประวัติส่วนตัว<PHONE>\n09</PHONE><PHO...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...,11-12: Exceptional – Rarely seen level of expe...,6.0
8,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...,**Evaluation:**\n\nScore: 7\nStrengths: \n- Th...,7.0
9,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...,Evaluation:\n\nScore: 6\nStrengths: \n- The ca...,6.0


## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


### Business Analysis

In [ ]:
# https://th.jobsdb.com/th/job/77991903?type=standout&ref=search-standalone&origin=jobCard#sol=1d2fd034b664b260b95e34d0a25df038709d37e2
ba = """
**Job Summary:**
Senior business analyst analyzes business processes on a large scale and devises strategies to improve efficiency. A senior business analyst evaluates the technicalities of a business problem and then shapes and crafts business solutions intuitively. They also review business processes to help diagnose the business operational and technological risks.

**Job Responsibilities:**
Develop requirement workshop plan and define requirement gathering approach.
Provide the end-to-end solution analysis by identifying business need, business constraints business impacts and new opportunities which may have in the future. Ensure that all requirements are compiled with applicable standard, BOT regulations and internal regulations.
Create and maintain business requirements document including business end to end workflow and data flow diagram which can be understand by business and technical team
Ensure quality of work products of team members and assign responsibility and allocate work target to the subordinated team to manage project achievements as planned
Evaluating how various project teams perform, and making sure they meet targets and deadlines

**Qualifications:**
Bachelor or master’s degree in Business Administration, Management, MIS, or related field (Master’s is preferred)
At least 5 years of experience in role of Business Analyst
Experience in developing business process and formal requirement documentation such as business requirement document
Experience with system or software development methodologies, including agile project management or agile development
Experience in project and team management
"""

In [ ]:
resume_prompt_ba = f"""
You are a highly experienced Human Resource Recruiter with a reputation for demanding excellence.
Your company, SCB TECH X CO., LTD. Only hires the best of the best, and you approach each resume with a critical eye, carefully weighing the value of each candidate.
As you review the resume below for a Senior IT Business Analyst Position , consider whether this candidate truly stands out in a competitive field.

{{resume}}
{ba}

**Evaluation:**

Score: [Your score here]
Strengths: [Identify the strengths of the resume]
Areas for Improvement: [Identify areas where the resume could be improved]
Overall Impression: [Summarize your overall thoughts on the resume]

**Scoring Scale:**

1-2: Very Poor – Significant gaps in skills or experience.
3-4: Poor – Some relevant skills but not meeting expectations.
5-6: Average – Meets basic requirements but lacks standout qualities.
7-8: Good – Strong qualifications but not exceptional.
9-10: Excellent – Outstanding and exceeds expectations.
"""

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient


# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        # Ensure the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Command to call Ghostscript
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",   # Output as PNG images
            f"-r{dpi}",           # Set the resolution (DPI)
            "-o", output_format,  # Output format
            pdf_file              # Input PDF file
        ]

        subprocess.run(gs_command, check=True)

        # Collect the generated image file paths
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files

    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)  # Enable GPU if available for faster processing
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])  # Sort based on y-coordinate
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to tag and clean the extracted text
def tag_and_clean_text(text):
    tagger = NamedEntityTagger()  # No device argument, as it uses GPU automatically if available
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    try:
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()
    except Exception as e:
        print(f"Error in tag_and_clean_text: {e}")
        return "", text

# Function to process a single PDF
def process_single_pdf(pdf_file):
    results = []
    try:
        # Convert PDF to images using Ghostscript
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        # Tag and clean text
        tagged_text, cleaned_text = tag_and_clean_text(raw_text)

        # Generate evaluation and score
        evaluation = generate_answer(cleaned_text)
        score = extract_score(evaluation)

        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text,
            "Evaluation": evaluation,
            "Score": score
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    # Convert results to a DataFrame
    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)  # Adjust the number as needed

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.5,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

# Function to generate an answer using the prompt
def generate_answer(resume):
    formatted_prompt = resume_prompt_ba.replace("{resume}", resume)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def extract_score(text):
    match = re.search(r'Score:\s*([0-9]+(?:\.[0-9]+)?)', text)
    if match:
        return float(match.group(1))
    return text

# Batch processing with threading
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    result = generate_answer(resume)
    return result

# Define the batch processing function
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)  # Initialize a list to store results in original order
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)  # Store the index range for each future

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

# Gradio Interface Function
def gradio_interface(pdf_files):
    try:
        df = process_multiple_pdfs(pdf_files)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Score'] = df['Evaluation'].apply(extract_score)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Gradio UI
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Files(label="Upload PDF Files")],  # Allow multiple files
    outputs=[
        gr.Dataframe(label="Extracted and Cleaned Data", height=500, min_width=800),
        gr.File(label="Download CSV")  # Add a file download button
    ],
    title="PDF to Text and Data Cleaner"
)

# Launch the interface
iface.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://37c675673aeb197cc1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://37c675673aeb197cc1.gradio.live


 > `7 mins`

In [ ]:
import pandas as pd
BA = pd.read_csv('/content/output.csv')
BA

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...,Evaluation:\n\nScore: 8\n\nStrengths: \n1. The...,8.0
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,Evaluation:\n\nScore: 6\nStrengths: \nThe cand...,6.0
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,Score: 7\n\nStrengths: \n1. The candidate has ...,7.0
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...,Evaluation:\n\nScore: 7\n\nStrengths: \nThe ca...,7.0
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,Evaluation:\n\nScore: 7.5\n\nStrengths: \n- Th...,7.5


## **Business Analysis Gradio**

In [5]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient

# Define the default job description placeholder
jobdescribe = """
**Job Summary:**
Senior business analyst analyzes business processes on a large scale and devises strategies to improve efficiency. A senior business analyst evaluates the technicalities of a business problem and then shapes and crafts business solutions intuitively. They also review business processes to help diagnose the business operational and technological risks.

**Job Responsibilities:**
Develop requirement workshop plan and define requirement gathering approach.
Provide the end-to-end solution analysis by identifying business need, business constraints business impacts and new opportunities which may have in the future. Ensure that all requirements are compiled with applicable standard, BOT regulations and internal regulations.
Create and maintain business requirements document including business end to end workflow and data flow diagram which can be understand by business and technical team
Ensure quality of work products of team members and assign responsibility and allocate work target to the subordinated team to manage project achievements as planned
Evaluating how various project teams perform, and making sure they meet targets and deadlines

**Qualifications:**
Bachelor or master’s degree in Business Administration, Management, MIS, or related field (Master’s is preferred)
At least 5 years of experience in role of Business Analyst
Experience in developing business process and formal requirement documentation such as business requirement document
Experience with system or software development methodologies, including agile project management or agile development
Experience in project and team management
"""

# Define the prompt with placeholders for resume and job description
resume_prompt = """
You are a highly experienced Human Resource Recruiter known for your unwavering commitment to excellence.
At your company, only the most exceptional candidates are considered, and you scrutinize each resume with a discerning eye.
You are tasked with evaluating this candidate against the detailed job description provided.

**Instructions:**
As you review the resume below for a [Position], consider whether this candidate truly differentiates themselves in a highly competitive field.
Rigorously assess their qualifications, skills, and experience in direct comparison to the specific requirements outlined in the job description.

{jobdescribe}

{resume}

Evaluation:

Score: [Your score here]
Strengths: [Identify the strengths of the resume]
Areas for Improvement: [Identify areas where the resume could be improved]
Overall Impression: [Summarize your overall thoughts on the resume]

**Scoring Scale:**
1-2: Very Poor – The candidate exhibits major deficiencies in skills or experience, making them highly unsuitable for the role.
3-4: Poor – The candidate shows some relevant skills but falls significantly short of the expected standards.
5-6: Average – The candidate meets the basic requirements but lacks the distinguishing qualities necessary to stand out.
7-8: Good – The candidate possesses strong qualifications and experience, yet may not demonstrate the exceptional caliber required for top consideration.
9-10: Excellent – The candidate is exemplary, exceeding all expectations and clearly standing out as a top contender for the role.
"""

In [14]:
a = pd.read_csv('/content/drive/MyDrive/AIEngineer/resume/promptba.csv')
a

,Unnamed: 0,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...,Score: 8\n\nStrengths:\n- The candidate has a ...,8.0
1,1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,Score: 8\n\nStrengths: \nThe candidate has a s...,8.0
2,2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,Score: 8\n\nStrengths:\n1. The candidate has a...,8.0
3,3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...,Experience:\nJob Objective:\nFinancial Analyst...,7.0
4,4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,Evaluation:\n\nScore: 8\nStrengths: \n1. The c...,8.0


In [13]:
print(a['Cleaned_Text'].iloc[3])

ประสบการณ์การ
เป้าหมายในการทำงาน
flnancial analyst
ต้องการเปลี่ยนสายงานจากการทำงานด้านการเงินมาสู่
 บริษัท finserve co. ltd.
business analyst โดยนำความเชี่ยวชาญใน
 ตำแหน่ง
การวิเคราะห์ทางการเงินมาประยุกต์ใช้ในการวิเคราะห์ธุรกิจ
 วิเคราะห์และจัดทำรายงานทางการเงินเพื่อ
เพื่อสนับสนุนการตัดสินใจขององค์กร
นำเสนอไห้กับผู้บริหาร
ประเมินความเสี่ยงและโอกาสทางการเงิน
ของโครงการลงทุนต่างฺๆ
ทำงานร่วมกับทีมบัญชีในการจัดทางบการ
เงินและวิเคราะห์ผลการดำเนินงาน
ทักษะ
accountant
 บริษัท global finance
 financial
sql
การวิเคราะห์ทางการเงิน
excel
 modeling
 จัดทำและตรวจสอบเอกสารทางการเงิน
การวิเคราะห์ธุรกิจ swot pestel cap analysis
รวมถึงการบันทึกบัญชีในระบบ
การใช้เครื่องมือ sap. quickbooks power bi
ทำรายงานภาษีและประสานงานกับหน่วย
การสื่อสาร การเขียนรายงาน ฺการนำเสนอข้อมูล
งานราชการ
 ให้คำปรึกษาด้านการเงินและภาษีกับลูกค้า
การศึกษา
 ปริญญาตรี สาขาการบัญชี
มหาวิทยาลัยหอการค้าไทย


In [8]:
# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to tag and clean the extracted text
def tag_and_clean_text(text):
    tagger = NamedEntityTagger()
    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    try:
        ner = tagger.get_ner(text, tag=True)
        pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
        cleaned_ner = re.sub(pattern, '', ner)
        cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
        cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner)
        cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)
        return ner.strip(), cleaned_ner.strip()
    except Exception as e:
        print(f"Error in tag_and_clean_text: {e}")
        return "", text

# Function to generate an answer using the prompt
def generate_answer(resume, job):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

# Function to extract score from the evaluation text
def extract_score(text):
    match = re.search(r'Score:\s*([0-9]+(?:\.[0-9]+)?)', text)
    if match:
        return float(match.group(1))
    return text

# Function to process a single PDF
def process_single_pdf(pdf_file, job):
    results = []
    try:
        images = convert_pdf_to_images(pdf_file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"

        tagged_text, cleaned_text = tag_and_clean_text(raw_text)
        evaluation = generate_answer(cleaned_text, job)
        score = extract_score(evaluation)

        results.append({
            "PDF File": os.path.basename(pdf_file.name),
            "Raw_Text": raw_text,
            "Tagged_Text": tagged_text,
            "Cleaned_Text": cleaned_text,
            "Evaluation": evaluation,
            "Score": score
        })

    except Exception as e:
        print(f"Error in process_single_pdf: {e}")

    df = pd.DataFrame(results)
    return df

# Function to process multiple PDFs
def process_multiple_pdfs(pdf_files, job):
    all_results = pd.DataFrame()
    for pdf_file in pdf_files:
        df = process_single_pdf(pdf_file, job)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Function to save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)

# Define the model parameters
model_params = {
    "max_new_tokens": 512,
    "temperature": 0.7,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job = row.get('Job_Description', "")
    result = generate_answer(resume, job)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

import gradio as gr

# Gradio Interface Function
def gradio_interface(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        # Update model parameters based on user input
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })

        df = process_multiple_pdfs(pdf_files, job)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Score'] = df['Evaluation'].apply(extract_score)
        csv_path = save_dataframe(df)
        return df, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""

# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column(scale=1):
                pdf_file_input = gr.Files(label="Upload PDF Files", file_count="multiple")
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)

                # Model parameters
                max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=512, step=50)
                temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.7, step=0.1)
                top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

                submit_btn = gr.Button("Process")

            with gr.Column(scale=2):
                output_df = gr.DataFrame(headers=["PDF File", "Raw_Text", "Tagged_Text", "Cleaned_Text", "Evaluation", "Score"], type="pandas", interactive=True)
                output_csv = gr.File(label="Download CSV")

            # Function to process inputs and generate output
            def on_submit(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty):
                df, csv_path = gradio_interface(pdf_files, job, max_new_tokens, temperature, top_p, repetition_penalty)
                return df, csv_path

            submit_btn.click(on_submit,
                             inputs=[pdf_file_input, job_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                             outputs=[output_df, output_csv])

    return demo

# Create and launch the Gradio interface
app = create_gradio_interface()
app.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c51a26dee11d282aa5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://c51a26dee11d282aa5.gradio.live


 > `7 mins`

In [ ]:
import pandas as pd
BA = pd.read_csv('/content/output.csv')
BA

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st\nเป้าหมายในการทำงาน\nนำความเชี่ยวชาญด้านการ...,Evaluation:\n\nScore: 8\n\nStrengths: \n1. The...,8.0
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ\n มุ่งมันในการนำทักษะด้านการจัดก...,Evaluation:\n\nScore: 6\nStrengths: \nThe cand...,6.0
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON> </PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม\n business analyst\nเป้าหมายในการทำงาน\n...,Score: 7\n\nStrengths: \n1. The candidate has ...,7.0
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON> </PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ\nเป้าหมายในการทำงาน\nflnancial a...,Evaluation:\n\nScore: 7\n\nStrengths: \nThe ca...,7.0
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทำงาน\nมุ่งมันที่จะใช้ความรู้และท...,Evaluation:\n\nScore: 7.5\n\nStrengths: \n- Th...,7.5


In [10]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Score
0,resume 6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON>\n</PERSON><PERSON>น</PERSON>...,in\n\n วิศวกรปัญญาประดิษฐ์\n ประสบการณ์การทำงา...,Based on the given resume and the provided job...,8.0
1,resume 2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON> วงศ์เจริญ\nณ็ชพล</PERSON>\n วิศวกรปัญ...,วิศวกรปัญญาประดิษฐ์\n ประวัติการศึกษา\nมหาวิทย...,11-12: Exceptional – The candidate not only me...,10.0
2,resume 3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume\nประวัติส่วนตัว<PHONE>\n09</PHONE><PHO...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี\nใน...,---\n\n## Job Description\n\nThe [Position] wi...,---\n\n## Job Description\n\nThe [Position] wi...
3,resume 4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON> </PERSON><PERSON>อาทิตย์\nรัตนวิจิตร<...,วิศวกรปัญญาประดิษฐ์\nประสบการณํ ทำงาน\nวิศวกร ...,Let's break down the evaluation based on the p...,Let's break down the evaluation based on the p...
4,resume 7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์\nประสบการณ์การฝึกงาน\nบัณฑ...,Evaluation:\n\nScore: 7\nStrengths: \n- The ca...,7.0
5,resume 1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์\nai ที่มีประสบการณ์ 3 ปีใน...,Evaluation:\n\nScore: 8\n\nStrengths: \n1. The...,8.0
6,resume 5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์\n ประสบการณ์ทำงาน\n\n วิศว...,To assess the candidate's resume for the AI En...,8.0


In [ ]:
BA.to_csv('/content/drive/MyDrive/AIEngineer/resume/promptba.csv')

### AI Engineer

In [ ]:
#https://th.jobsdb.com/th/job/78442339?type=standout&ref=search-standalone&origin=jobCard#sol=1c90597dae76a0f00f141b14d0cba749bda3167d
ai = """
**Job Summary:**
The senior AI engineer position will be part of the technical labs team of SCB 10X. Some responsibilities include but are not limited to

**Job Descriptions:**
Be the main person to lead AI activities and development for the lab team
Support the Venture Capital team to evaluate prospective companies, particularly on their AI technology capability and feasibility.
Stay up to date on the latest technology emergences relevant to the business domain.
Conduct research and development processes for AI strategies.
Collaborate with cross-functional teams to explore ideas where AI could be developed into any promising business.
Provide technical support in product idea validation.
Work with industrial experts, other engineers, and scientists to develop a proof of concept of validated ideas.
Contributing to AI communities through some of these activities: knowledge sharing, technical blog writing, etc.

**Qualifications:**
Experience in formulating and strategizing AI solutions.
Hands-on knowledge in AI, machine learning, deep learning, PyTorch, Python, NLP.
Experience in one or more specialized machine learning areas, e.g., NLP, deep learning, recommendation systems, computer vision, conversational AI, and reinforcement learning.
Proven experience in applying AI to practical and all-inclusive technology solutions.
Knowledge of various deep learning model architectures.
Strong ability to modify deep learning model architectures.
Experience with training deep learning models on large datasets.
Strong programming skills in Python and proficiency with relevant libraries such as Numpy, Pandas, and Matplotlib.
Strong team player: prioritize work and organize your time to support others efficiently.
Excellent analytical thinking and problem-solving skills.
Excellent written and verbal communication skills.
Self-starter and fast learner in working with new areas/topics.
Solid computer science fundamentals in statistical learning theory, algorithms, data structures are preferred.
Deep knowledge of one or more industrial domains is a plus.
"""